In [27]:
%pip install requests

You should consider upgrading via the 'c:\Users\fenpon\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


In [28]:
import re
import pandas as pd
import requests


In [29]:


def get_coordinates_kakao(address, api_key):
    url = "https://dapi.kakao.com/v2/local/search/address.json"
    headers = {"Authorization": f"KakaoAK {api_key}"}
    params = {"query": address}
    
    response = requests.get(url, headers=headers, params=params)
    
    if response.status_code == 200:
        result = response.json()
        if result["documents"]:
            location = result["documents"][0]["address"]
            if location is None:
                return None, None
            return float(location["y"]), float(location["x"])  # 위도, 경도 반환
        else:
            print(f"해당 주소에 대한 좌표 정보를 찾을 수 없습니다. {address}")
            return None, None
    else:
        print( f"에러 발생: {response.status_code}")
        return None, None


In [30]:
#병원 점수 예측 데이터
trainedHospital = pd.read_csv('../export/safety/policeFacilities.csv')


# 주소 컬럼에서 괄호 안의 내용과 괄호를 제거
trainedHospital['주소'] = trainedHospital['주소'].str.replace(r'\(.*?\)', '', regex=True)

# 결과 출력
print(trainedHospital['주소'])

0               서울특별시 중구  을지로 234
1             서울특별시 중구  퇴계로 375-1
2      서울특별시 중구  동호로 5길 15, 약수지구대
3              서울특별시 중구  다산로 248 
4               서울특별시 중구  동호로 261
                  ...            
238            서울특별시 강남구  삼성로 203
239          서울특별시 강남구  밤고개로1길 48
240            서울특별시 강남구  헌릉로 623
241     서울특별시 강남구  일원로3길 35 일원파출소
242            서울특별시 강남구  개포로 261
Name: 주소, Length: 243, dtype: object


In [31]:




block = 0
le = 0
# 사용 예제


for i in range(len(trainedHospital)):
    address = trainedHospital.iloc[i]['주소']
    address = address.split(' ')
    address = ' '.join(address[:4])
    coord = get_coordinates_kakao(address,'a4fef1a0b1958ed22e57f606e5fad553')
    trainedHospital.at[i,'lat'] = coord[0]
    trainedHospital.at[i,'lon'] = coord[1]

    block += 1
    if block > 100:
        print("--- " ,block , "  : ",le)
        block = 0
        le += 1

---  101   :  0
---  101   :  1


In [32]:
trainedHospital.to_csv('../export/safety/policeFacilityOutput.csv', index=False, encoding='utf-8') 
